In [1]:
import findspark
findspark.init('/home/yerke/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
my_data = spark.read.format('libsvm').load('../course_data/sample_libsvm_data.txt')

In [6]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
my_log_reg_model = LogisticRegression()

In [9]:
fitted_logreg = my_log_reg_model.fit(my_data)

In [10]:
log_summary = fitted_logreg.summary

In [12]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [14]:
lr_train, lr_test = my_data.randomSplit([.7,.3])

In [15]:
final_model = LogisticRegression()

In [17]:
fit_final = final_model.fit(lr_train)

In [18]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [20]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[23.9289525503281...|[0.99999999995946...|       0.0|
|  0.0|(692,[124,125,126...|[17.2286549267434...|[0.99999996706253...|       0.0|
|  0.0|(692,[124,125,126...|[20.4824518319728...|[0.99999999872771...|       0.0|
|  0.0|(692,[125,126,127...|[33.1320556248874...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[43.0223218979918...|           [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[29.6879794709932...|[0.99999999999987...|       0.0|
|  0.0|(692,[127,128,129...|[23.6251327274204...|[0.99999999994507...|       0.0|
|  0.0|(692,[151,152,153...|[39.8892335943530...|           [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|[32.5576209604531...|[0.99999999999999...|       0.0|
|  0.0|(692,[154

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [22]:
my_eval = BinaryClassificationEvaluator()

In [23]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [24]:
my_final_roc

1.0